# Introductort Mars Express and ExoMars2016 Example

This is a Python Jupyter Notebook to illustrate a SPICE running example. 
This could be your very own Python 3 environment, the first thing we will do is indicate that we want the Python package matplotlib to be output in the notebook and to import the SpiceyPy package to use SPICE.

In [2]:
%matplotlib inline

import math
import spiceypy as spiceypy

## A Mars Express Phobos observation example

In this first example, we will compute some basic geometry for a Phobos observation by the HRSC camera, more concretely the SRC sensor. We an retrieve the image from the Planetary Science Archive (PSA) User Interface. 

![title](img/HF780_0004_SR2.JPG)

The metadata of the image is as follows:

In [3]:
spiceypy.furnsh('/Users/mcosta/MARS-EXPRESS/kernels/mk/MEX_OPS_LOCAL.TM')

We convert from UTC to Ephemeris Time (ET) the time of the image: 2016-06-14T03:11:56.218Z

In [4]:
et = spiceypy.utc2et('2016-06-14T03:11:56.208')

We can check that the SCLK information provided at the product corresponds to the one provided by the SPICE Kernels

In [5]:
id = spiceypy.bodn2c('MARS EXPRESS')
sclk = spiceypy.sce2s(id,et,32)

print(sclk)

1/0413953910.47849


We obtain the HRSC boresight and boresight reference frame

In [6]:
et = spiceypy.utc2et('2016-06-14T03:11:56.218')

sensor_name = 'MEX_HRSC_SRC'

sensor_id = spiceypy.bodn2c(sensor_name)
(shape, frame, bsight, vectors, bounds) = spiceypy.getfov(sensor_id, 100)

print('{} shape: {}'.format(sensor_name, shape))
print('{} frame: {}'.format(sensor_name, frame))
print('{} bsight: {}'.format(sensor_name, bsight))
print('{} vectors: {}'.format(sensor_name, vectors))
print(bounds)

MEX_HRSC_SRC shape: RECTANGLE
MEX_HRSC_SRC frame: MEX_HRSC_SRC
MEX_HRSC_SRC bsight: [  0.     0.   984.76]
MEX_HRSC_SRC vectors: 4
[[  4.64050347   4.64050347 984.73813222]
 [ -4.64050347   4.64050347 984.73813222]
 [ -4.64050347  -4.64050347 984.73813222]
 [  4.64050347  -4.64050347 984.73813222]]


We obtain the intersection between the boresight and Phobos

In [7]:
(spoint, trgepc, srfvec ) = spiceypy.sincpt('ELLIPSOID', 'PHOBOS', et, 'IAU_PHOBOS', 'NONE', 'MEX', frame, bsight)

SpiceyError: Spice returns not found for function: sincpt

Of course, it does not work because we are currently not intersecting the body, so let's look for an moment in time in which there is interception. We can simply modify our vector in the HRSC SRC frame:

In [8]:
bsight = [-4.64050347,  0.0, 984.73813222]

(spoint, trgepc, srfvec ) = spiceypy.sincpt('ELLIPSOID', 'PHOBOS', et, 'IAU_PHOBOS', 'NONE', 'MEX', frame, bsight)

Finally we compute the illumination angles

In [9]:
(trgepc, srfvec, phase, solar, emissn) = spiceypy.ilumin('ELLIPSOID', 'PHOBOS', et, 'IAU_PHOBOS', 'NONE', 'MEX', spoint)

print('Phase Angle: {} [DEG], Solar Incidence: {} [DEG]'.format(math.degrees(phase), math.degrees(solar)))

Phase Angle: 55.72661057620171 [DEG], Solar Incidence: 64.0710850675298 [DEG]


But we could load a Digital Shape Model of PHOBOS and assess the difference in illumination by using the shape model. The DSK that we load is as follows:

![title](img/phobos_dsk.png)

In [11]:
spiceypy.furnsh('/Users/mcosta/Dropbox/SPICE/SPICE_CROSS_MISSION/epsc2017-workshop/lessons/remote_sensing_tgo/kernels/dsk/phobos_2014_09_22.bds')

(spoint, trgepc, srfvec ) = spiceypy.sincpt('DSK/UNPRIORITIZED', 'PHOBOS', et, 'IAU_PHOBOS', 'NONE', 'MEX', frame, bsight)
(trgepc, srfvec, phase, solar, emissn) = spiceypy.ilumin('DSK/UNPRIORITIZED', 'PHOBOS', et, 'IAU_PHOBOS', 'NONE', 'MEX', spoint)

print('Phase Angle: {} [DEG], Solar Incidence: {} [DEG]'.format(math.degrees(phase), math.degrees(solar)))

Phase Angle: 55.726610770196245 [DEG], Solar Incidence: 61.88293185629522 [DEG]


Which compared to the value with the reference Ellipsoid:

And this is what we would see in Cosmographia:
    
![title](img/cosmo.png)


Now, we will show the object oriented capabilities of sciops, aimed to easy the way that we interface with SPICE. 
We will define a Time Window, Phobos as a target and Mars Express as an observer:

In [14]:
import spiops as spiops

interval = spiops.TimeWindow('2016-11-26T00:07:15', '2016-11-26T10:21:32', resolution=10)

phobos = spiops.Target('PHOBOS', time=interval, frame='IAU_PHOBOS')
mex = spiops.Observer('MEX', time=interval, target=phobos)

The spiops library will plot some geometric quantites at our desire

In [15]:
mex.Plot('distance', notebook=True)

Loading BokehJS ...